## Observations and Insights 

In [1]:
#1 When reviewing the information you can see that the most successful drug regimens are
#..Capomulin and Ramicane. This is observed by looking at the box and wnisker plot and seeing 
#..that the tumor volume at the end of the treatment was much smaller than the other drugs it 
#..was compared to. 
#2  The line plot used for the Capomulin drug treatment showed a significant decrese in tumor 
#..volume over time. 
#3 After completing the linear regression, we can see that the heavier the mouse,
#..the higher the tumor volume. 

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import scipy.stats as linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)



FileNotFoundError: [Errno 2] File data/Mouse_metadata.csv does not exist: 'data/Mouse_metadata.csv'

In [ ]:
# Combine the data into a single dataset
mouse_data = "Data/Mouse_metadata.csv"
study_data = "Data/Study_results.csv"
mouse_data_df = pd.read_csv(mouse_data)
study_data_df = pd.read_csv(study_data)
#mouse_data_df
#study_data_df

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint.


In [ ]:
combined_data = pd.merge(mouse_data_df,study_data_df, how="left", on="Mouse ID")
# Display the data table for preview
combined_data

In [ ]:
# Checking the number of mice.
combined_data["Mouse ID"].value_counts()


In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
total_unique_mice = combined_data["Mouse ID"].unique()
count_unique_mice = pd.DataFrame(total_unique_mice).count()
count_unique_mice

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
dup_mice = combined_data.loc[combined_data.duplicated(subset = ["Mouse ID","Timepoint"]),"Mouse ID"].unique()
#dup_mice
# Checking the number of mice in the clean DataFrame.
clean_df = combined_data[combined_data["Mouse ID"].isin(dup_mice)==False]
clean_df.head()

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
dup_mice_df = combined_data[combined_data["Mouse ID"].isin(dup_mice)==True]
dup_mice_df.head()

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
mean = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean()
median = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].median()
variance = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].var()
stdv = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].std()
sem = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].sem()

# This method is the most straighforward, creating multiple series and putting them all together at the end.
summary_df = pd.DataFrame({"Mean": mean, "Median": median, "Variance": variance, "Standard Deviation": stdv, 
                          "SEM": sem})
summary_df


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summary_stats = combined_data.groupby("Drug Regimen")["Tumor Volume (mm3)"].describe()
summary_stats
# This method produces everything in a single groupby function


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
grouped_df = pd.DataFrame(clean_df.groupby(["Drug Regimen"]).count()).reset_index()
regimen = grouped_df[["Mouse ID","Drug Regimen"]]
regimen = regimen.set_index("Drug Regimen")
regimen.plot(kind="bar")
plt.show()


In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.
drugs = summary_stats.index.tolist()
#drugs
regimen_count = (clean_df.groupby(["Drug Regimen"])["Age_months"].count())
regimen_count


In [ ]:
x_axis = np.arange(len(regimen_count))
x_axis = drugs
plt.bar(x_axis, regimen_count, alpha=1,align="center")
plt.xticks(rotation="vertical")
plt.ylabel("Number of Data Points")
plt.xlabel("Drug Regimen")

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
gender_pie_pd =pd.DataFrame(clean_df.groupby(["Sex"]).count()).reset_index()
gender_pie_pd


In [ ]:
gender_pie_pd = gender_pie_pd[["Sex","Mouse ID"]]
#gender_pie_pd
gender_pie_pd.plot(kind="pie",y="Mouse ID", autopct="%1.1f%%", labels=gender_pie_pd["Sex"], title="Sex", legend=False)

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
gender_pie_py = (clean_df.groupby(["Sex"])["Age_months"].count()).tolist()
#gender_pie_py
labels = ["Females","Males"]
colors = ["orange","blue"]
plt.pie(gender_pie_py, labels=labels,colors=colors,autopct="%1.1f%%",startangle=140)
plt.axis("equal")
plt.title("Sex")

## Quartiles, Outliers and Boxplots

In [ ]:
clean_df.columns

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
# Start by getting the last (greatest) timepoint for each mouse
# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
cap_df = clean_df.loc[clean_df["Drug Regimen"] == "Capomulin",:]
ram_df = clean_df.loc[clean_df["Drug Regimen"] == "Ramicane", :]
inf_df = clean_df.loc[clean_df["Drug Regimen"] == "Infubinol", :]
cef_df = clean_df.loc[clean_df["Drug Regimen"] == "Ceftamin", :]
#cap_df
#ram_df
#inf_df
#cef_df

In [ ]:
#Capomulin greatest timepoint 
cap_last = cap_df.groupby("Mouse ID").max()["Timepoint"]
#cap_last
cap_vol = pd.DataFrame(cap_last)
cap_merge = pd.merge(cap_vol, clean_df, on=("Mouse ID","Timepoint"),how="left")
cap_merge.head()

In [ ]:
#Ramicane greatest timepoint
ram_last = ram_df.groupby("Mouse ID").max()["Timepoint"]
#ram_last
ram_vol = pd.DataFrame(ram_last)
ram_merge = pd.merge(ram_vol, clean_df, on=("Mouse ID","Timepoint"),how="left")
ram_merge.head()

In [ ]:
#Infubinol greatest timepoint 
inf_last = inf_df.groupby("Mouse ID").max()["Timepoint"]
inf_last
inf_vol = pd.DataFrame(inf_last)
inf_merge = pd.merge(inf_vol, clean_df, on=("Mouse ID","Timepoint"),how="left")
inf_merge.head()

In [ ]:
#Ceftamin greatest timepoint 
cef_last = cef_df.groupby("Mouse ID").max()["Timepoint"]
#cap_last
cef_vol = pd.DataFrame(cef_last)
cef_merge = pd.merge(cef_vol, clean_df, on=("Mouse ID","Timepoint"),how="left")
cef_merge.head()

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine if there are any potential outliers 
# Determine outliers using upper and lower bounds
#Capomulin
cap_tumors = cap_merge["Tumor Volume (mm3)"]
cap_tumors
quartiles = cap_tumors.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of Capomulin tumor treatment is: {lowerq}")
print(f"The upper quartile of Capomulin tumor treatment is: {upperq}")
print(f"The interquartile range of Capomulin tumor treatment is: {iqr}")
print(f"The the median of Capomulin tumor treatment is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")


In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine if there are any potential outliers 
# Determine outliers using upper and lower bounds
#Ramicane
ram_tumors = ram_merge["Tumor Volume (mm3)"]
ram_tumors
quartiles = ram_tumors.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of Ramicane tumor treatment is: {lowerq}")
print(f"The upper quartile of Ramicane tumor treatment is: {upperq}")
print(f"The interquartile range of Ramicane tumor treatment is: {iqr}")
print(f"The the median of Ramicane tumor treatment is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine if there are any potential outliers 
# Determine outliers using upper and lower bounds
#Infubinol
inf_tumors = inf_merge["Tumor Volume (mm3)"]
inf_tumors
quartiles = inf_tumors.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of Infubinol tumor treatment is: {lowerq}")
print(f"The upper quartile of Infubinol tumor treatment is: {upperq}")
print(f"The interquartile range of Infubinol tumor treatment is: {iqr}")
print(f"The the median of Infubinol tumor treatment is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
# Determine if there are any potential outliers 
# Determine outliers using upper and lower bounds
#Ceftamin
cef_tumors = cef_merge["Tumor Volume (mm3)"]
cef_tumors
quartiles = cef_tumors.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of Ceftamin tumor treatment is: {lowerq}")
print(f"The upper quartile of Ceftamin tumor treatment is: {upperq}")
print(f"The interquartile range of Ceftamin tumor treatment is: {iqr}")
print(f"The the median of Ceftamin tumor treatment is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
# Create empty list to fill with tumor vol data (for plotting)
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
    # Locate the rows which contain mice on each drug and get the tumor volumes
    # add subset     

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

tumor_data = [cap_tumors, ram_tumors, inf_tumors, cef_tumors]
Regimen= ['Capomulin', 'Ramicane', 'Infubinol','Ceftamin']

fig1, ax1 = plt.subplots()
ax1.set_ylabel('Final Tumor Volume (mm3)')
ax1.set_xlabel('Drug')
ax1.boxplot(tumor_data, labels=Regimen)

plt.ylim(15, 75)

plt.show()

## Line and Scatter Plots

In [ ]:
cap_df

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
cap_line = cap_df.loc[cap_df["Mouse ID"]=="l509",:]
cap_line

In [ ]:
x_axis = cap_line["Timepoint"]
y_axis = cap_line["Tumor Volume (mm3)"]
plt.title("Capomulin treatment of mouse l509")
plt.plot(x_axis, y_axis)
plt.xlabel("Timepoint (days)")
plt.ylabel("Tumor Volume (mm3)")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
avg_cap = pd.DataFrame(cap_df.groupby(["Mouse ID", "Weight (g)"])["Tumor Volume (mm3)"].mean()).reset_index()
avg_cap = avg_cap.rename(columns={"Tumor Volume (mm3)":"Average Tumor Volume (mm3)"})
avg_cap.head()

In [ ]:
plt.scatter(avg_cap["Weight (g)"],avg_cap["Average Tumor Volume (mm3)"])
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
cap_weight = cap_df.groupby(["Mouse ID"])["Weight (g)"].mean()
#cap_weight
cap_tumor = cap_df.groupby(["Mouse ID"])["Tumor Volume (mm3)"].mean()
#cap_tumor
print(f"The correlation coefficient between mouse weight and average tumor volume is {round(st.pearsonr(cap_weight,cap_tumor)[0],2)}")

In [ ]:
from scipy.stats import linregress

x_values = avg_cap["Weight (g)"]
y_values = avg_cap["Average Tumor Volume (mm3)"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show()
